# Deploying with Actions and Terraform

### Introduction

In the last lesson, we saw how we can use an identity provider to login to AWS from Github Actions.  In this lesson, we'll go further.

Before moving on, let's think about what we would like to accomplish.  In other words, when we make a commit, what are the steps that we will need to occur to when we make some change in our code, and push up our code to github.  

Try to write them down now.

### Developing a plan

Ok, so we'll want the following to occur:

1. Run our tests, and if our tests pass
2. Rebuild our image
3. Push the image up to ECR
4. Build the terraform infrastructure (EC2 and security groups)
5. Pull down the docker image onto our EC2 instance

### Rebuilding the image

Let's skip ahead to rebuilding our Docker image.  Ok, so if you look at our code, you'll see that the first step is to log in to our ecr repository.

Let's take some time to unpack the following.

```bash
- name: Build, tag, and push image to Amazon ECR
      env:
        ECR_REGISTRY: ${{ steps.login-ecr.outputs.registry }}
        ECR_REPOSITORY: flask_app_sample
        IMAGE_TAG: latest
      working-directory: app
      run: |
        docker build -t $ECR_REGISTRY/$ECR_REPOSITORY:$IMAGE_TAG .
        docker push $ECR_REGISTRY/$ECR_REPOSITORY:$IMAGE_TAG
```

At the very top, we use `env` to declare two environmental variables into our github actions environment: `ECR_REGISTRY` and `ECR_REPOSITORY`.

* `ECR_REGISTRY` Here we get the value, from the previous step where we logged into ECR for our AWS account.  Note that we labeled that step with an id, `id: login-ecr`.  And that logging in to ECR, provides a registry value in the format of `account-num.dkr.ecr.aws-region.amazonaws.com`.  This is the ECR registry where all of your repositories are stored.  You can imagine that the `login-ecr` step provides a dictionary of outputs, and that we just retrieved the registry value, and assigned it to `ECR_REGISTRY`.

```python
outputs = {'registry': 'account-num.dkr.ecr.aws-region.amazonaws.com'}
```

Ok, so then the only other component that's needed is the specific repository you would like to push the code to.  

* Create a repository that matches your `ECR_REPOSITORY` value, so you can push up to there.

Ok, so once we are logged into ECR, and AWS knows what repository to push to, think of the steps that we want to accomplish next.

1. CD into the directory that has our docker file
    * `working-directory: app` accomplishes this
2. Build and push our repository to the ECR repo
    * The lines in `run:` accomplish this.
    * The `|` allows you to use a multiline string in github actions
    * And then we are building and pushing our image, referencing the environmental variables set with `env`

### Setting up terraform

Ok, so now that we have code that will build and push our docker repo, the next step is to set up code that will initialize our terraform infrastructure.

Notice that we have the following steps:
```yaml
    - name: Setup Terraform
      uses: hashicorp/setup-terraform@v1
      with:
        terraform_version: 1.0.0
    - name: Terraform Init
      working-directory: tf
      run: terraform init

    - name: Terraform Plan
      working-directory: tf
      run: terraform plan

    - name: Terraform Apply
      working-directory: tf
      run: terraform apply -auto-approve
```

* Setup Terraform The first is a setup terraform step, that sets up the terraform cli in github actions with `uses: hashicorp/setup-terraform@v1`
* Init, Plan, and Apply
    * Next we move through our normal steps of calling `terraform init`, `terraform plan` and `terraform apply` to deploy our changes.
    * **Notice**: At each step we set the `working-directory` to `tf`.  This is because, we need to be in the `tf` directory before calling each of these commands -- just like we would if we were on our laptop.

* Terraform destroy

If something breaks in our process of deploying, we don't want to have things just lying around.  So we call `terraform destroy` if there is any failure.

```yaml
- name: Terraform Destroy
      if: failure()
      run: terraform destroy -auto-approve
```

> The auto-approve is because terraform will otherwise ask us to type `yes` to proceed.

### The Terraform Code

If you take a look at our terraform code, you can see what occurs when we boot up our ec2 instance. 

```bash
user_data = <<-EOF
  #!/bin/bash
  export AWS_ACCOUNT_ID=$(aws sts get-caller-identity --query "Account" --output text)
  export REGION=us-east-1
  export BACKEND_CONTAINER=flask_api
  export REPOSITORY_NAME=flask_app_sample
  sudo yum update -y
  sudo yum install docker -y
  sudo systemctl start docker
  ...
  ```

Ok, so the in user data we move through the normal steps of setting up docker.  Perhaps the most interesting part is the line:
    
```bash
export AWS_ACCOUNT_ID=$(aws sts get-caller-identity --query "Account" --output text)
```

Here, the goal is to use the `aws` command line to get our account_id.  This way we can use that account id when pulling our image from our ECR later on, as in:

```bash
sudo docker pull $AWS_ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPOSITORY_NAME:latest
```

Ok, so we get the account id, with the `sts get-caller-id` which returns an output in the format of:
    
```bash
{"UserId": "AIDACKCEVSQ6C2EXAMPLE","Account": "123456789012", "Arn": "arn:aws:iam::123456789012:user/JohnDoe"}
```
And so we query the `Account` property and store it as `AWS_ACCOUNT_ID`.

The rest of the above code just downloads and sets up docker.  And then further on, we move through the steps of logging our EC2 machine into our ECR to download our ECR.  And then pulling the correct repository and booting up the image.

```bash
aws ecr get-login-password --region $REGION | sudo docker login --username AWS --password-stdin $AWS_ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com
  sudo docker pull $AWS_ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPOSITORY_NAME:latest

  while ! sudo docker container ls | grep -wq $BACKEND_CONTAINER; do
    sudo docker run -d -p 80:80 --name $BACKEND_CONTAINER --platform=linux/amd64/v2 $AWS_ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPOSITORY_NAME
    sleep 5
  done
```

#### Making it Work for You

So to get this code to work for you, you'll need to go to the `tf/main.tf` file, and in that `user_data` section, you'll need to update the `region`, and `repository_name` (to match the repository name that you want to pull from in ECR).

Afterwards, you should be able to push to github actions, and watch it work.

<img src="./actions-apply.png" width="60%">

And then you can go to your aws web console to find the EC2 that you just launched, as well as the domain name to connect to.

Go to `http://domain_name` (not `https`) and you should see your website.

> <img src="./working-container.png" width="50%">

### What's next

Ok, so we did develop a github actions pipeline that will build our docker image, push it to ecr, setup our terraform infrastructure, and pull down our images, and boot up a container.

However, in production we would have a problem in that each time we push to github actions, a new ec2 machine would be created.  And there's not really to run `terraform destroy` to delete or update our infrastructure, after our github actions pipeline has completed.

The problem is that to reference the infrastructure that was just created, terraform needs the `terraform.tfstate` file -- and that is on your github actions environment.  So the next step would be to store our `terraform.tfstate` file in an S3 bucket, and use that file when calling our terraform commands.  

> Check it out: You can see how to do this in the [following blog post](https://faun.pub/deploy-to-aws-ec2-using-terraform-and-github-actions-ci-cd-bfcb60b3cc1e).

* One other issue

Once we do that, we would also want to make sure our `user_data` script that pulls down our docker image from ECR is run each time we update our web application code, even if we do not update our terraform code.  To make sure that script is run, one trick is to place a timestamp in the `user_data` code, this way we ensure our terraform code changes, and thus our user_data code is re-run each time.

```bash
locals {
  timestamp = timestamp()
}

resource "aws_instance" "example" {
  ami           = "ami-123456"
  instance_type = "t2.micro"

  user_data = <<-EOF
                #!/bin/bash
                echo "Deployed at: ${local.timestamp}"
                ...
```

### Resources

[Next Steps - CI CD and TF](https://faun.pub/deploy-to-aws-ec2-using-terraform-and-github-actions-ci-cd-bfcb60b3cc1e)